In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import datetime
import re

In [2]:
def get_next_page(driver):
    try:
        driver.find_element_by_class_name("ui_button.nav.next.primary").send_keys(Keys.ENTER)
        return True
    except:
        print('마지막 페이지입니다')
        return False

In [3]:
def convert_date(date):
    stay_date = date.split('숙박 날짜: ')
    stay_date = stay_date[1]

    # 20XX년 X월 형태의 str
    stay_date = stay_date.replace('년', '').replace(' ', '').replace('월', '')
    dt = datetime.datetime.strptime(stay_date, '%Y%m')
    dt = dt.date() #20XX-XX-01 형태의 date 객체
    return dt

In [23]:
# convert_date 실행 예시
convert_date('숙박 날짜: 2020년 11월')

datetime.date(2020, 11, 1)

In [4]:
def get_data(driver):
    iteration = True
    df = pd.DataFrame(columns = ['nickname', 'score', 'review', 'stay_date'])
    try:
        time.sleep(20)
        #WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 
        #                                                                    "div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu")))
        # 더보기 클릭
        driver.find_element_by_css_selector("div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu").click()

        time.sleep(5)
        # 숙박 날짜 가져오기
        dates = driver.find_elements_by_class_name("_34Xs-BQm")
        stay_dates = []
        for d in dates:
            stay_date = convert_date(d.text)
            stay_dates.append(stay_date)
            # 날짜 수정 - 코로나 전 1년 반 - 코로나 후 1년 반
            if stay_date < datetime.date(2018,6,1): 
                iteration = False

        # 닉네임 가져오기
        nicknames = driver.find_elements_by_css_selector("div._310S4sqz > div > div._2fxQ4TOx > span > a")
        # 별점 가져오기
        rating_codes = driver.find_elements_by_css_selector("div.oETBfkHU > div._2UEC-y30 > div > span")
        scores = []
        for rc in rating_codes:
            cls_attr = str(rc.get_attribute("class")).split("ui_bubble_rating bubble_")
            score = int(cls_attr[1])
            scores.append(score)

        # 리뷰 가져오기
        reviews = driver.find_elements_by_css_selector("div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu > div.cPQsENeY > q > span")
        
        for i in range(len(nicknames)):
            df.loc[i] = [nicknames[i].text, scores[i], reviews[i].text, stay_dates[i]]
            #print(df.loc[i])

    except ElementClickInterceptedException as e:
        print(e)
    except StaleElementReferenceException as e:
        print(e)

    finally:
        return df, iteration

In [5]:
def hotel_review_crawling(url):
    # 데이터 수집
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(options=options)

    data = pd.DataFrame(columns = ['nickname', 'score', 'review', 'stay_date'])

    #첫페이지 시작
    #데이터 크롤링, df에 추가하기
    #다음페이지 클릭 -> disbaled 나오기 전까지 반복

    driver.get(url)
    driver.implicitly_wait(10)
    iteration = True
    page = 1
    while iteration:
        print(page, '번째 페이지')
        tmp_df, iteration = get_data(driver)
        data = pd.concat([data, tmp_df], axis=0, ignore_index=True)
        if iteration == False:
            break
        iteration = get_next_page(driver) 
        page += 1

    #data.to_csv('./hotel_review_test_sillastay_mapo.csv', encoding="utf-8-sig")
    
    return data

In [6]:
def preprocessing_data(data):
    # 개행문자 제거
    for idx, review in enumerate(data['review']):
        data['review'][idx] = review.replace('\n', ' ')
    # 날짜순으로 내림차순 정렬
    data.sort_values(by='stay_date', ascending=False, 
                     ignore_index=True, inplace=True)
    # 날짜 수정 - 코로나 전 1년 반 - 코로나 후 1년 반
    data = data[data['stay_date'] > datetime.date(2018,5,1)]
    # 특수문자, 이모지 제거
    ###
    return data

In [7]:
# 실제로 실행하는 코드
url = "https://www.tripadvisor.co.kr/Hotel_Review-g297885-d8093030-Reviews-Best_Western_Jeju_Hotel-Jeju_Jeju_Island.html"

data_sample = hotel_review_crawling(url)
data_sample

1 번째 페이지
2 번째 페이지
3 번째 페이지
4 번째 페이지
5 번째 페이지
6 번째 페이지
7 번째 페이지
8 번째 페이지
9 번째 페이지
10 번째 페이지
11 번째 페이지
12 번째 페이지
13 번째 페이지
14 번째 페이지
15 번째 페이지
16 번째 페이지
17 번째 페이지
18 번째 페이지
19 번째 페이지
20 번째 페이지
21 번째 페이지
22 번째 페이지
23 번째 페이지
24 번째 페이지
25 번째 페이지
26 번째 페이지
27 번째 페이지
28 번째 페이지
29 번째 페이지
30 번째 페이지
31 번째 페이지
32 번째 페이지
33 번째 페이지
34 번째 페이지
35 번째 페이지
36 번째 페이지
37 번째 페이지
38 번째 페이지
39 번째 페이지
40 번째 페이지
41 번째 페이지
42 번째 페이지
43 번째 페이지
44 번째 페이지
45 번째 페이지
46 번째 페이지
47 번째 페이지
48 번째 페이지
49 번째 페이지
50 번째 페이지
51 번째 페이지
52 번째 페이지
53 번째 페이지
54 번째 페이지
55 번째 페이지
56 번째 페이지
57 번째 페이지
58 번째 페이지
59 번째 페이지
60 번째 페이지
61 번째 페이지
62 번째 페이지
63 번째 페이지
64 번째 페이지
65 번째 페이지
66 번째 페이지
67 번째 페이지
68 번째 페이지
69 번째 페이지
70 번째 페이지
71 번째 페이지
72 번째 페이지
73 번째 페이지
74 번째 페이지
75 번째 페이지


,nickname,score,review,stay_date
0,Explore00773435628,50,제주공항 근처에 위치해있고 노형동 맛집과도 거리가 가까워서 첫날이나 마지막날 숙소로...,2021-06-01
1,Carlos88,50,제주여행 오면서\n이곳저곳 많이 다녀봤는데\n이만한 호텔~~~가성비 짱이네요\n공항...,2021-05-01
2,로하마미,40,아이가 있어 일부러 키즈룸으로 예약했는데 아이가 너무 좋아하네요~ 1층에 스벅있는건...,2021-04-01
3,Straw7,50,오전 뱅기라 공항 근처로 했는데 신제주 지역이라 주변에 맛집도 많고 좀 걸어서 이마...,2021-05-01
4,Lee,50,마지막 날이라 공항과 가까운곳에 묵었는데 이마트도 가깝고 너무 좋네요. 객실도 깔끔...,2021-04-01
...,...,...,...,...
370,뚠때니,50,"접근성 굉장히 좋고 GS25, 스타벅스 건물내에 있어용 호텔 내에 스타벅스를 이용할...",2018-07-01
371,JH Choi,50,2박 위해 더블룸 예약했습니다. 비교적 저렴했구요. 도착했는데 1층에 스타벅스와 수...,2018-07-01
372,yooo1357,50,객실도 깔끔하고 공항과 가까워서 이동하기 편리했습니다.\n직원들도 친절히 설명해주어...,2018-06-01
373,kingishes,50,제주 공항과 가깝고 깔끔하고 가성비 좋은 호텔입니다 여기에 더불어 조식도 끝내주고요...,2018-06-01


In [9]:
# 기본 전처리 전 원본 데이터 저장(유실방지)
data_sample.to_csv('./data/bestwestern_jeju_origin.csv', encoding="utf-8-sig")

In [10]:
data = preprocessing_data(data_sample)

In [11]:
data

,nickname,score,review,stay_date
0,Explore00773435628,50,제주공항 근처에 위치해있고 노형동 맛집과도 거리가 가까워서 첫날이나 마지막날 숙소로...,2021-06-01
1,Straw7,50,오전 뱅기라 공항 근처로 했는데 신제주 지역이라 주변에 맛집도 많고 좀 걸어서 이마...,2021-05-01
2,Carlos88,50,제주여행 오면서 이곳저곳 많이 다녀봤는데 이만한 호텔~~~가성비 짱이네요 공항가깝고...,2021-05-01
3,로하마미,40,아이가 있어 일부러 키즈룸으로 예약했는데 아이가 너무 좋아하네요~ 1층에 스벅있는건...,2021-04-01
4,Lee,50,마지막 날이라 공항과 가까운곳에 묵었는데 이마트도 가깝고 너무 좋네요. 객실도 깔끔...,2021-04-01
...,...,...,...,...
369,Jay L,50,"매전 출장올때마다 이곳만 이용하는데, 깔끔하고 친절한 서비스에 항상 만족하고있어요~...",2018-07-01
370,본 리,40,깨끗하고 펍도 있어서 좋아요 ^^ 방도 넒고 시설도 괜찮네요~ 프론트 직원들도 너무...,2018-07-01
371,KilsuKim,40,"호텔에 도착하면, 먼저 입구에 비치된 시원한 물한잔을 마십니다. 프론트 데스크 직원...",2018-07-01
372,yooo1357,50,객실도 깔끔하고 공항과 가까워서 이동하기 편리했습니다. 직원들도 친절히 설명해주어서...,2018-06-01


In [12]:
# csv 저장을 위한 date to str convert
data['stay_date'] = data['stay_date'].apply(lambda X : X.strftime('%Y-%m-%d'))
# 기본 전처리후 데이터 csv 파일로 저장 
#data.to_csv('./data/hotel_review_shillastay_yeoksam.csv', encoding="utf-8-sig")

<ipython-input-12-7183346d9643>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['stay_date'] = data['stay_date'].apply(lambda X : X.strftime('%Y-%m-%d'))


In [13]:
# 리뷰 내 이모지 제거
data['review'] = data['review'].apply(lambda X : X.encode('euc-kr', 'ignore').decode('euc-kr'))
#data.to_csv('./data/hotel_review_shillastay_yeoksam_del_emoji.csv', encoding="utf-8-sig")

<ipython-input-13-b71e843350f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['review'] = data['review'].apply(lambda X : X.encode('euc-kr', 'ignore').decode('euc-kr'))


In [14]:
# 리뷰 내 특수문자 제거
test_data = data
special = re.compile(r'[^ A-Za-z0-9가-힣+]')
test_data['review'] = [special.sub('', s) for s in test_data['review']]

# 특수문자 제거 후 파일 저장
test_data.to_csv('./data/bestwestern_jeju_final.csv', encoding="utf-8-sig")

<ipython-input-14-46474791da4d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['review'] = [special.sub('', s) for s in test_data['review']]


In [15]:
test_data

,nickname,score,review,stay_date
0,Explore00773435628,50,제주공항 근처에 위치해있고 노형동 맛집과도 거리가 가까워서 첫날이나 마지막날 숙소로...,2021-06-01
1,Straw7,50,오전 뱅기라 공항 근처로 했는데 신제주 지역이라 주변에 맛집도 많고 좀 걸어서 이마...,2021-05-01
2,Carlos88,50,제주여행 오면서 이곳저곳 많이 다녀봤는데 이만한 호텔가성비 짱이네요 공항가깝고 중심...,2021-05-01
3,로하마미,40,아이가 있어 일부러 키즈룸으로 예약했는데 아이가 너무 좋아하네요 1층에 스벅있는건 ...,2021-04-01
4,Lee,50,마지막 날이라 공항과 가까운곳에 묵었는데 이마트도 가깝고 너무 좋네요 객실도 깔끔하...,2021-04-01
...,...,...,...,...
369,Jay L,50,매전 출장올때마다 이곳만 이용하는데 깔끔하고 친절한 서비스에 항상 만족하고있어요 욕...,2018-07-01
370,본 리,40,깨끗하고 펍도 있어서 좋아요 방도 넒고 시설도 괜찮네요 프론트 직원들도 너무 친절...,2018-07-01
371,KilsuKim,40,호텔에 도착하면 먼저 입구에 비치된 시원한 물한잔을 마십니다 프론트 데스크 직원에게...,2018-07-01
372,yooo1357,50,객실도 깔끔하고 공항과 가까워서 이동하기 편리했습니다 직원들도 친절히 설명해주어서 ...,2018-06-01


In [17]:
df = pd.read_csv('./data/glad_yeouido_final.csv')
df

,Unnamed: 0,nickname,score,review,stay_date
0,0,주희 이,50,서비스 시설 모든게 완벽했습니다 특히 객실 내에 깔끔한 분위기와 휴식과 비즈니스 모...,2021-07-01
1,1,hwan jun jang,50,여의도 glad 호텔 grace님 체크인 잘 안내해주시고 친절하시고 좋았습니다 호텔...,2021-07-01
2,2,박은희,50,출장와서 급하게 예약을 한다고 룸컨디션도 안보고 했는데 로비에서 만난 케빈님이 친절...,2021-06-01
3,3,Pizza__,50,기대는 안했었는데 방도 넓고 욕실도 깨끗하고 너무 좋습니다 예약한 룸보다 업그레이...,2021-06-01
4,4,김세화,50,여의도 글래드 호텔 침구가 좋기로 유명한데 역시 침구도 최고고 데스크에 계신 직원분...,2021-06-01
...,...,...,...,...,...
1129,1129,hella663,50,시내에 위치하고 있어 접근성이 좋았다 차로도 편했고 지하철역이 바로 앞에 있어 대중...,2018-06-01
1130,1130,Julia H,50,가까운 곳에서 편하게 하루 쉴 만한 곳을 찾았는데 아주 편하게 쉬다 갑니다 너무 힘...,2018-06-01
1131,1131,yeyekims,50,방이 생각보다 넓고 침구가 아주 편해서 좋아용 그리고 호텔 전체가 이쁘고 모던합니다...,2018-06-01
1132,1132,정인 박,50,우선 직원들의 응대가 친절하고 정확하며 빠릅니다 룸 컨디션은 쾌적하고 깨끗하며 너무...,2018-06-01


In [18]:
df = df[['nickname', 'score', 'review', 'stay_date']]

In [19]:
final = pd.concat([df, test_data], axis=0, ignore_index=True)
final

,nickname,score,review,stay_date
0,주희 이,50,서비스 시설 모든게 완벽했습니다 특히 객실 내에 깔끔한 분위기와 휴식과 비즈니스 모...,2021-07-01
1,hwan jun jang,50,여의도 glad 호텔 grace님 체크인 잘 안내해주시고 친절하시고 좋았습니다 호텔...,2021-07-01
2,박은희,50,출장와서 급하게 예약을 한다고 룸컨디션도 안보고 했는데 로비에서 만난 케빈님이 친절...,2021-06-01
3,Pizza__,50,기대는 안했었는데 방도 넓고 욕실도 깨끗하고 너무 좋습니다 예약한 룸보다 업그레이...,2021-06-01
4,김세화,50,여의도 글래드 호텔 침구가 좋기로 유명한데 역시 침구도 최고고 데스크에 계신 직원분...,2021-06-01
...,...,...,...,...
1503,Jay L,50,매전 출장올때마다 이곳만 이용하는데 깔끔하고 친절한 서비스에 항상 만족하고있어요 욕...,2018-07-01
1504,본 리,40,깨끗하고 펍도 있어서 좋아요 방도 넒고 시설도 괜찮네요 프론트 직원들도 너무 친절...,2018-07-01
1505,KilsuKim,40,호텔에 도착하면 먼저 입구에 비치된 시원한 물한잔을 마십니다 프론트 데스크 직원에게...,2018-07-01
1506,yooo1357,50,객실도 깔끔하고 공항과 가까워서 이동하기 편리했습니다 직원들도 친절히 설명해주어서 ...,2018-06-01


In [21]:
final.to_csv('./data/testset1.csv', encoding="utf-8-sig")